In [ ]:
import re
import os
import sys
import glob
import pickle
import shelve
import numpy as np
from scipy.fft import fft, fftfreq
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.transforms as mtransforms
from matplotlib.colors import LogNorm, FuncNorm
from matplotlib.gridspec import GridSpec
from matplotlib.ticker import FixedLocator, NullLocator, FixedFormatter
import seaborn as sns
from datetime import datetime

fontsize = 7
lw = 0.75

matplotlib.rc('font', **{'family': 'Times New Roman', 'size': fontsize})
matplotlib.rc('axes', **{'linewidth': 0.75, 'labelsize': fontsize})
matplotlib.rc('xtick', **{'labelsize': fontsize})
matplotlib.rc('ytick', **{'labelsize': fontsize})
matplotlib.rc('xtick.major', **{'width': lw, 'size':3})
matplotlib.rc('ytick.major', **{'width': lw, 'size':3})
matplotlib.rc('ytick.minor', **{'width': lw, 'size':1.5})

%matplotlib inline

if '..' not in sys.path:
    sys.path.append('..')
from dlml.data import load_data_areas, load_data_slide
from dlml.utils import collect_experiments

In [ ]:
def plot_correlations(R_mean, R_ctrl_mean, edges, ax, sort_freq=1.0,
                      vmin=None, vmax=None, legend_bbox=[0.4, -0.05]):
    edge = np.abs(edges - sort_freq).argmin()
    kdx = np.argsort(R_mean[edge,:])
    R_mean = R_mean[:,kdx]
    R_ctrl_mean = R_ctrl_mean[:,kdx]

    make_symmetric = False
    if vmin is None:
        vmin = min([r.min() for r in R_mean])
        make_symmetric = True
    if vmax is None:
        vmax = max([r.max() for r in R_mean])
        if make_symmetric:
            if vmax > np.abs(vmin):
                vmin = -vmax
            else:
                vmax = -vmin
    print(f'Color bar bounds: ({vmin:.2f},{vmax:.2f}).')
    ticks = np.linspace(vmin, vmax, 7)
    ticklabels = [f'{tick:.2f}' for tick in ticks]

    cmap = plt.get_cmap('bwr')
    x = np.arange(R_mean.shape[-1])
    y = edges[:-1] + np.diff(edges) / 2
    im = ax[0].pcolormesh(x, y, R_mean, vmin=vmin, vmax=vmax, shading='auto', cmap=cmap)
    i = 0
    ax[i].set_xticks(np.linspace(0, x[-1], 3, dtype=np.int32))
    if len(ax) == 3:
        i += 1
        ax[i].pcolormesh(x, y, R_ctrl_mean, vmin=vmin, vmax=vmax, shading='auto', cmap=cmap)
        ax[i].set_xticks(np.linspace(0, x[-1], 3, dtype=np.int32))
    cbar = plt.colorbar(im, fraction=0.1, shrink=1, aspect=20, label='Correlation',
                        orientation='vertical', ax=ax[i], ticks=ticks)
    cbar.ax.set_yticklabels(ticklabels, fontsize=fontsize-1)
    R_abs_mean = np.mean(np.abs(R_mean), axis=1)
    R_ctrl_abs_mean = np.mean(np.abs(R_ctrl_mean), axis=1)
    ax[i+1].plot(R_abs_mean, y, 'r', lw=1, label='Tr.')
    ax[i+1].plot(R_ctrl_abs_mean, y, 'g--', lw=1, label='Untr.')
    ax[i+1].plot(R_abs_mean - R_ctrl_abs_mean, y, 'k', lw=1, label='Diff.')
    ax[i+1].legend(loc='lower left', bbox_to_anchor=legend_bbox, frameon=False, fontsize=fontsize-1)

    for a in ax:
        a.set_ylim(edges[[0,-2]])
        a.set_yscale('log')

In [ ]:
def plot_subplots(N, edges, norm_std, F, Xf, exact_momentum, pred_momentum, pred_momentum_ctrl,
                  MAPE, R_mean, R_ctrl_mean, corr_edges, ax):
    target_values = np.unique(exact_momentum)
    
    tab10 = plt.get_cmap('tab10')
    green, magenta = tab10(2), tab10(6)
    cmap = lambda i: (green, magenta)[i%2]

    for i in range(2):
        ax[0].plot(edges[i][1:], N[i], lw=1, color=cmap(i))
        ax[1].plot(F, 20*np.log10(Xf[i]), lw=1, color=cmap(i))

    ax[0].grid(which='major', axis='x', lw=0.5, ls=':', color=[.6,.6,.6])
    ax[0].set_xlim([-4.5, 4.5])
    ax[0].set_xticks(np.r_[-4 : 4.5 : 2])
    ax[0].set_ylabel('PDF')
    ax[0].set_xlabel('Normalized V')
    ticks = np.r_[0 : 0.61 : 0.2]
    ax[0].set_ylim(ticks[[0,-1]])
    ax[0].yaxis.set_major_locator(FixedLocator(ticks))
    ax[0].yaxis.set_major_formatter(FixedFormatter([f'{tick:g}' for tick in ticks]))
    for i in range(2):
        ax[0].text(-4, 0.6-i*0.1*np.diff(ax[0].get_ylim()), f'STD={norm_std[i]:.2f}', color=cmap(i),
                   fontsize=fontsize-1, va='top')

    ax[1].set_ylim([-55, -10])
    ax[1].set_yticks(np.r_[-50 : -9 : 10])
    ax[1].set_xscale('log')
    ax[1].set_xlabel('Frequency [Hz]')
    ax[1].set_ylabel('Power [dB]')
    f_ticks = np.array([0.1, 0.2, 0.5, 1, 2, 5, 10, 20])
    ax[1].set_xlim(f_ticks[[0,-1]] + np.array([0,1]))
    ax[1].xaxis.set_major_locator(FixedLocator(f_ticks))
    ax[1].xaxis.set_minor_locator(NullLocator())
    ax[1].xaxis.set_major_formatter(FixedFormatter([f'{tick:g}' for tick in f_ticks]))
    ax[1].grid(which='major', axis='y', lw=0.5, ls=':', color=[.6,.6,.6])
    for i,v in enumerate(target_values):
        ax[1].text(0.12, -48-i*0.1*np.diff(ax[1].get_ylim()),
                   r'M={:.2f} GW$\cdot$s$^2$'.format(v),
                   color=cmap(i), fontsize=fontsize-1)

    df = pd.DataFrame(data={'Exact': exact_momentum, 'Pred': np.concatenate(pred_momentum)})
    df_ctrl = pd.DataFrame(data={'Exact': exact_momentum, 'Pred': np.concatenate(pred_momentum_ctrl)})
    sns.violinplot(x='Exact', y='Pred', data=df, cut=0, inner='quartile',
                   palette=[cmap(0), cmap(1)], ax=ax[2], linewidth=0.5)
    ax[2].xaxis.set_major_locator(FixedLocator([0, 1]))
    ax[2].xaxis.set_major_formatter(FixedFormatter([f'{tick:.2f}' for tick in target_values]))
    ax[2].yaxis.set_major_locator(FixedLocator(target_values))
    ax[2].yaxis.set_major_formatter(FixedFormatter([f'{tick:.2f}' for tick in target_values]))
    ax[2].text(np.mean(ax[2].get_xlim()), target_values.mean(), f'MAPE={MAPE:.1f}%',
               ha='center', va='center', fontsize=fontsize-1)

    plot_correlations(R_mean, R_ctrl_mean, corr_edges, sort_freq=[1.1], ax=ax[3:])
    for i in (3,4):
        ax[i].set_ylim(f_ticks[[0,-1]] + np.array([0,1]))
        ax[i].yaxis.set_major_locator(FixedLocator(f_ticks))
        ax[i].yaxis.set_minor_locator(NullLocator())
        if i == 3:
            ax[i].yaxis.set_major_formatter(FixedFormatter([f'{tick:g}' for tick in f_ticks]))
        else:
            ax[i].set_yticklabels([])
    ax[3].set_ylabel('Frequency [Hz]')
    ax[3].set_xlabel('Filter #')
    ax[4].set_xlabel('Correlation')

    sns.despine()

## Figure S1

In [ ]:
find_best_experiment_IDs = False

if find_best_experiment_IDs:
    from dlml.utils import collect_experiments
    area_measure = 'momentum'
    stoch_load_bus_IDs = []
    rec_bus_IDs = [3]
    H_G1, D, DZA = None, None, None # 500, 2, 0
    best_experiment_IDs = {1: {'Vd_bus3': '474d2016e33b441889ce8b17531487cb',
                               'Vq_bus3': '617188cbcaef4816a8853081fd303ac1'}}
    for area_ID in (1,2):
        if area_ID not in best_experiment_IDs:
            best_experiment_IDs[area_ID] = {}
        for var_name in (f'Vd_bus{rec_bus_IDs[0]}', f'Vq_bus{rec_bus_IDs[0]}'):
            if var_name not in best_experiment_IDs[area_ID]:
                additional_tags = ['ReLU_none', 'converted_from_PowerFactory', 'all_stoch_loads',
                                   var_name.split('_')[0]]
                expts = collect_experiments(area_ID, area_measure=area_measure, D=D, DZA=DZA, \
                                            stoch_load_bus_IDs=stoch_load_bus_IDs, H_G1=H_G1, \
                                            rec_bus_IDs=rec_bus_IDs, additional_tags=additional_tags, \
                                            verbose=False)
                if expts is None or len(expts) == 0:
                    continue
                expt_IDs = list(expts.keys())
                expt_ID = expt_IDs[np.argmin([expt['val_loss'].min() for expt in expts.values()])]
                MAPE = expts[expt_ID]['MAPE']
                loss = expts[expt_ID]['loss']
                val_loss = expts[expt_ID]['val_loss']
                batch_loss = expts[expt_ID]['batch_loss']
                tags = expts[expt_ID]['tags']
                best_experiment_IDs[area_ID][var_name] = expt_ID
                print(f'The best experiment is {expt_ID[:6]} ' + \
                      f'(val_loss = {val_loss.min():.4f}, ' + \
                      f'MAPE = {MAPE:.4f}%).')
else:
    best_experiment_IDs = {1: {'Vd_bus3': '474d2016e33b441889ce8b17531487cb',
                               'Vq_bus3': '617188cbcaef4816a8853081fd303ac1'}, #'4c8bfb605ed74ea2935f3c3ce416b52d'
                           2: {'Vd_bus3': 'fb6e5dd5df00455fb12c97d0daf77d84',
                               'Vq_bus3': 'f19eac813b484b8c8708ebde2cd5b2c0'}}

In [ ]:
rows = 4
fig,ax = plt.subplots(rows, 5, width_ratios=[1, 1.8, 1, 2.2, 1], figsize=(16.5/2.54, 3.75/2.54*rows))

force = False
k = 0
for area_ID in (1,2):
    for var_name in ('Vd_bus3','Vq_bus3'):
        experiment_ID = best_experiment_IDs[area_ID][var_name]
        data_file = f'hist_spectra_acc_corr_area_{area_ID}_{var_name}_{experiment_ID[:6]}.npz'
        if force or not os.path.isfile(data_file):
            set_name = 'training'
            data_dir = '/home/daniele/Research/deep-power/data/IEEE39/converted_from_PowerFactory/' + \
                f'all_stoch_loads/var_H_area_{area_ID}_comp_grid/subset_2'
            data_files = sorted(glob.glob(os.path.join(data_dir, '*' + set_name + '*.h5')))
            generators_areas_map = [['G02','G03','Comp11'],
                             ['G04','G05','G06','G07','Comp21'],
                             ['G08','G09', 'G10','Comp31'],
                             ['G01']]
            generators_Pnom = {'G01': 10000e6, 'G02': 700e6, 'G03': 800e6, 'G04': 800e6, 'G05': 300e6,
                        'G06': 800e6, 'G07': 700e6, 'G08': 700e6, 'G09': 1000e6, 'G10': 1000e6,
                        'Comp11': 100e6, 'Comp21': 100e6, 'Comp31': 100e6}

            ret = load_data_areas({set_name: data_files}, [var_name],
                                  [generators_areas_map[ID-1] for ID in [area_ID]],
                                  generators_Pnom,
                                  area_measure='momentum',
                                  trial_dur=60,
                                  max_block_size=10000,
                                  use_tf=False,
                                  add_omega_ref=True,
                                  use_fft=False)

            t = ret[0]
            X_raw = ret[1][set_name]
            y = ret[2][set_name]
            group_index = [np.where(y == mom)[0] for mom in np.unique(y)]
            n_mom_groups = len(group_index)
            X_mean, X_std = X_raw.mean(axis=(1,2)), X_raw.std(axis=(1,2))
            X = (X_raw - X_mean) / X_std
            X = X.squeeze()
            y = y.squeeze()
            norm_std = [X[idx].std() for idx in group_index]

            dt = np.diff(t[:2])[0]
            N_samples = t.size
            Xf = fft(X)
            Xf = 2.0 / N_samples * np.abs(Xf[:, :N_samples//2])
            F = fftfreq(N_samples, dt)[:N_samples//2]

            Xf = [Xf[idx,:].mean(axis=0) for idx in group_index]
            N,edges = zip(*(np.histogram(X[idx,:], bins=50, density=True) for idx in group_index))

            experiments_path = '../experiments/neural_network/'
            test_results = pickle.load(open(os.path.join(experiments_path, experiment_ID,
                                                         'test_results.pkl'), 'rb'))
            MAPE = test_results['mape_prediction'][0]
            N_bands = 60
            filter_order = 6
            if experiment_ID == '474d2016e33b441889ce8b17531487cb':
                N_trials = 4000
            else:
                N_trials = 1000
            correlations_file = f'correlations_{experiment_ID[:6]}_{N_bands}-bands_64-filters_' + \
                f'36-neurons_{N_trials}-trials_{filter_order}-butter_{var_name}_pool_1_3.npz'
            correlations = np.load(os.path.join(experiments_path, experiment_ID, correlations_file))
            R, R_ctrl = correlations['R'], correlations['R_ctrl']
            R[correlations['p'] > 0.05] = np.nan
            R_ctrl[correlations['p_ctrl'] > 0.05] = np.nan
            R_mean = np.nanmean(R, axis=0)
            R_ctrl_mean = np.nanmean(R_ctrl, axis=0)

            np.savez_compressed(data_file, N=N, edges=edges, F=F, Xf=Xf, norm_std=norm_std,
                                exact_momentum=correlations['exact_momentum'],
                                pred_momentum=correlations['pred_momentum'],
                                pred_momentum_ctrl=correlations['pred_momentum_ctrl'], MAPE=MAPE,
                                R_mean=R_mean, R_ctrl_mean=R_ctrl_mean, corr_edges=correlations['edges'])

        data = np.load(data_file)
        for key in data.files:
            exec(f'{key} = data["{key}"]')

        plot_subplots(N, edges, norm_std, F, Xf, exact_momentum, pred_momentum, pred_momentum_ctrl,
              MAPE, R_mean, R_ctrl_mean, corr_edges, ax[k,:])
        ax[k,2].set_title('Area {}, {}'.format(area_ID, var_name.replace('_',' @ ')), fontsize=fontsize+1)
        xl,yl = ax[k,3].get_xlim(), ax[k,3].get_ylim()
        x,y = xl[0] + np.diff(xl)/20, np.logspace(np.log10(yl[0]), np.log10(yl[1]), 20)[1]
        ax[k,3].text(x, y, experiment_ID[:6], fontsize=fontsize+1, color='k')
        k += 1

only_first = True
if only_first:
    trans = mtransforms.ScaledTranslation(-0.4, -0.05, fig.dpi_scale_trans)
else:
    trans = mtransforms.ScaledTranslation(-0.3, -0.05, fig.dpi_scale_trans)
for i,label in enumerate('ABCD'):
    if only_first:
        ax[i,0].text(0.0, 1.0, label, transform=ax[i,0].transAxes + trans, fontsize=fontsize+2, va='bottom')
    else:
        for j in range(5):
            ax[i,j].text(0.0, 1.0, label+str(j+1), transform=ax[i,j].transAxes + trans,
                         fontsize=fontsize, va='bottom')
fig.tight_layout(pad=0.3)
plt.savefig('hist_spectra_acc_corr_supp.pdf')

## Figure S2

In [ ]:
experiment_ID = '474d2016e33b441889ce8b17531487cb'
N_bands = 20, 40, 60
rows = len(N_bands)
filter_order = 6
N_trials = 4000
experiments_path = '../experiments/neural_network'
var_name = 'Vd_bus3'

offset = np.array([[0.11, 0.1], [0.07, 0.025]])
space = [0.12, 0.05]
rows, cols = 3, 3
height = (1 - offset[:,1].sum() - space[1]*(rows-1)) / rows
width = (1 - offset[:,0].sum() - space[0]*(cols-1)) / (cols+2)
fig = plt.figure(figsize=(9/2.54, 3/2.54*rows))
ax = []
for i in range(rows):
    ax.append([])
    for j in range(cols):
        x = offset[0,0] + j*(space[0] + width*2)
        if j == 1:
            x -= space[0]/1.5
        y = 1 - offset[1,1] - (i+1)*height - i*space[1]
        if j == 0:
            w = 2 * width
        elif j == 1:
            w = 2.1 * width
        else:
            w = width
        h = height
        a = plt.axes([x, y, w, h])
        ax[i].append(a)
ax = np.array(ax)

for i,N in enumerate(N_bands):
    correlations_file = f'correlations_{experiment_ID[:6]}_{N}-bands_64-filters_' + \
        f'36-neurons_{N_trials}-trials_{filter_order}-butter_{var_name}_pool_1_3'
    if not os.path.isfile(os.path.join(experiments_path, experiment_ID, correlations_file + '_small.npz')):
        correlations = np.load(os.path.join(experiments_path, experiment_ID, correlations_file + '.npz'))
        R, R_ctrl = correlations['R'], correlations['R_ctrl']
        R[correlations['p'] > 0.05] = np.nan
        R_ctrl[correlations['p_ctrl'] > 0.05] = np.nan
        R_mean = np.nanmean(R, axis=0)
        R_ctrl_mean = np.nanmean(R_ctrl, axis=0)
        data = {'R_mean': R_mean, 'R_ctrl_mean': R_ctrl_mean}
        for k in 'edges', 'exact_momentum', 'pred_momentum', 'pred_momentum_ctrl':
            data[k] = correlations[k]
        np.savez_compressed(os.path.join(experiments_path, experiment_ID,
                                         correlations_file + '_small.npz'), **data)
    data = np.load(os.path.join(experiments_path, experiment_ID, correlations_file + '_small.npz'))
    plot_correlations(data['R_mean'], data['R_ctrl_mean'], data['edges'], sort_freq=[1.1], ax=ax[i,:])
    f_ticks = np.array([0.1, 0.2, 0.5, 1, 2, 5, 10, 20])
    for j in range(3):
        ax[i,j].set_ylim(f_ticks[[0,-1]] + np.array([0,1]))
        ax[i,j].yaxis.set_major_locator(FixedLocator(f_ticks))
        ax[i,j].yaxis.set_minor_locator(NullLocator())
        if j != 1:
            ax[i,j].yaxis.set_major_formatter(FixedFormatter([f'{tick:g}' for tick in f_ticks]))
        else:
            ax[i,j].set_yticklabels([])
        if i != rows-1:
            ax[i,j].set_xticklabels([])
    ax[i,0].set_ylabel('Frequency [Hz]')
    ax[i,1].text(60, 10, f'# bands: {N}', fontsize=fontsize+1, ha='right')

ax[-1,-1].set_xlabel('Correlation')
ax[-1,0].set_xlabel('Filter #')
ax[-1,1].set_xlabel('Filter #')
sns.despine()

trans = mtransforms.ScaledTranslation(-0.35, -0.05, fig.dpi_scale_trans)
for i,label in enumerate('ABC'):
    ax[i,0].text(0.0, 1.0, label, transform=ax[i,0].transAxes + trans, fontsize=fontsize+2, va='bottom')
        
plt.savefig('corrs_vs_Nbands_supp.pdf')

## Figure S3

In [ ]:
# this function computes the output size of a preprocessing pipeline given
# its kernel size(s), kernel stride(s) and number of units in the pooling layer
def compute_output_size(N_inputs, kernel_size, kernel_stride, pool_size, verbose=False):
    f = lambda n_inp, k_sz, k_strd, pool_sz: ((n_inp - k_sz) // k_strd + 1) // pool_sz
    n_char = int(np.ceil(np.log10(N_inputs)))
    fmt = '{{:{0}d}} -> {{:{0}d}}'.format(n_char)
    for i,(ksz,kstr,ps) in enumerate(zip(kernel_size,kernel_stride,pool_size)):
        N_outputs = f(N_inputs, ksz, kstr, ps)
        if verbose: print(fmt.format(N_inputs, N_outputs))
        N_inputs = N_outputs
    return N_outputs

In [ ]:
trial_dur = 60  # [s]
srate = 40      # [Hz]
N_samples = trial_dur * srate
units = {'conv': [16, 32, 64], 'pool': [4, 4, 4], 'dense': [64]}
kernel = {'sizes': [5, 5, 5], 'strides': [1, 1, 1]}
dfs = []

kernel_sizes = 3, 5, 7, 9
kernel_strides = 1, 2
pool_sizes = 2, 4, 6, 8

metrics = 'MAPE','val_loss','loss','batch_loss'
time_interval = [datetime(2023, 4, 4), datetime(2023, 4, 7)]
output_size = {}
for kernel_size in kernel_sizes:
    kernel['sizes'] = [kernel_size for _ in range(len(kernel['sizes']))]
    output_size[kernel_size] = {}
    for kernel_stride in kernel_strides:
        kernel['strides'] = [kernel_stride for _ in range(len(kernel['strides']))]
        output_size[kernel_size][kernel_stride] = {}
        for pool_size in pool_sizes:
            units['pool'] = [pool_size for _ in range(len(units['pool']))]
            out_sz = compute_output_size(N_samples, kernel['sizes'], kernel['strides'], units['pool'])
            output_size[kernel_size][kernel_stride][pool_size] = out_sz
            if out_sz == 0:
                print('Combination (kernel_size,kernel_stride,pool_size) = ({},{},{}) is unfeasible'.
                     format(kernel_size, kernel_stride, pool_size))
                continue
            additional_tags = ['ReLU_none', 'converted_from_PowerFactory', 'all_stoch_loads',
                               'data_subset', 'Vd', 'trial_dur_{}'.format(trial_dur)]
            for k,v in units.items():
                tag = 'N_' + k + '_units_' + '_'.join(map(str, v))
                additional_tags.append(tag)
            for k,v in kernel.items():
                tag = 'kernel_' + k + '_' + '_'.join(map(str, v))
                additional_tags.append(tag)
            experiments = collect_experiments(area_IDs=[1],
                                              area_measure='momentum',
                                              rec_bus_IDs=[3],
                                              additional_tags=additional_tags,
                                              time_interval=time_interval,
                                              full_metrics=False, verbose_level=0)
            if experiments is None:
                print('No experiments with (pool_size,kernel_size,kernel_stride) = ({},{},{})'.
                     format(pool_size, kernel_size, kernel_stride))
                continue
            experiment_IDs = list(experiments.keys())
            metric_values = np.array([[experiments[ID][metric] for ID in experiment_IDs]
                                      for metric in metrics]).T
            rows,_ = np.where(np.isnan(metric_values))
            idx = np.setdiff1d(np.arange(metric_values.shape[0]), np.unique(rows))
            n = idx.size
            data = {'pool_size':     [pool_size     for _ in range(n)],
                    'kernel_size':   [kernel_size   for _ in range(n)],
                    'kernel_stride': [kernel_stride for _ in range(n)],
                    'output_size':   [out_sz        for _ in range(n)]}
            for i,metric in enumerate(metrics):
                data[metric] = metric_values[idx,i]
            df = pd.DataFrame(data=data, index=[experiment_IDs[i] for i in idx])
            dfs.append(df)
df = pd.concat(dfs)

In [ ]:
rows = 1
cols = 2

fig,ax = plt.subplots(rows, cols, figsize=(6.5*cols/2.54, 5*rows/2.54), sharex=True, squeeze=False)

for i in range(cols):
    stride = i+1
    sns.violinplot(data=df[df.kernel_stride==stride], x='kernel_size', y='val_loss',
                   hue='pool_size', palette='Set2', cut=0, inner='quartile', scale='count',
                   linewidth=0.75, ax=ax[0,i])
    if rows > 1:
        sns.violinplot(data=df[df.kernel_stride==stride], x='kernel_size', y='MAPE',
                       hue='pool_size', palette='Set2', cut=0, inner='quartile', scale='count',
                       linewidth=0.75, ax=ax[1,i])
    ax[0,i].set_title(f'Stride = {i+1}', fontsize=fontsize+2)

sns.despine()

ax[0,0].set_ylabel('Validation loss')
if rows == 1:
    for i in range(cols):
        ax[0,i].set_xlabel('Kernel size')
    if cols == 2:
        ax[0,1].legend_.set_visible(False)
        ax[0,1].set(ylabel=None)
    lgnd = ax[0,0].legend_
else:
    for i in range(cols):
        ax[0,i].legend_.set_visible(False)
        ax[0,i].set(xlabel=None)
        ax[1,i].set_xlabel('Kernel size')
    if cols == 2:
        for i in range(rows):
            ax[i,1].set(ylabel=None)
        ax[1,1].legend_.set_visible(False)
    lgnd = ax[1,0].legend_
    ax[1,0].set_ylabel('MAPE [%]')
lgnd.set_frame_on(False)
lgnd.set_title('Pool size')
lgnd.set_bbox_to_anchor((0.85, 0.5))

if rows == 1:
    ticks = np.logspace(np.log10(0.002), np.log10(0.02), 7)
else:
    ticks = np.logspace(np.log10(0.002), np.log10(0.02), 4)
for i in range(cols):
    ax[0,i].set_yscale('log')
    ax[0,i].set_ylim(ticks[[0,-1]] + np.array([-0.0001, 0.001]))
    ax[0,i].yaxis.set_major_locator(FixedLocator(ticks))
    ax[0,i].yaxis.set_minor_locator(NullLocator())
    if i == 0:
        ax[0,i].yaxis.set_major_formatter(FixedFormatter([f'{tick:.3f}' for tick in ticks]))
    else:
        ax[0,i].set_yticklabels([])

if rows > 1:
    ticks = np.logspace(np.log10(0.5), np.log10(9), 4)
    for i in range(cols):
        ax[1,i].set_yscale('log')
        ax[1,i].set_ylim(ticks[[0,-1]] + np.array([-0.05, 0.5]))
        ax[1,i].yaxis.set_major_locator(FixedLocator(ticks))
        if i == 0:
            ax[1,i].yaxis.set_major_formatter(FixedFormatter([f'{tick:.0f}' for tick in ticks]))
        else:
            ax[1,i].set_yticklabels([])

for i in range(rows):
    for j in range(cols):
        ax[i,j].grid(which='major', axis='y', ls=':', lw=0.5, color=[.6,.6,.6])

fig.tight_layout()
plt.savefig(f'hyperpars_opt_supp.pdf')

### Table S1

In [ ]:
with open('hyperpars_table.tex', 'w') as fid:
    fid.write('\\begin{table}[tb!]\n')
    fid.write('\\begin{center}\n')
    fid.write('\\caption{Hyperparameters impact on \\ac{cnn} accuracy\n')
    fid.write('\\label{tab:hyperpars}}\n')
    fid.write('\\begin{tabular}{ccccc}\n')
    fid.write('\\toprule\n')
    fid.write('\\thead{Kernel size} & \\thead{Kernel stride} & \\thead{Pooling size} & \n')
    fid.write('\\thead{Output size} & \\thead{Median val. loss (n=10)} \\\\\n')
    fid.write('\\midrule\n')
    for j,psz in enumerate(pool_sizes):
        for i,ksz in enumerate(kernel_sizes):
            for kstrd in kernel_strides:
                out_sz = output_size[ksz][kstrd][psz]
                if out_sz > 0:
                    idx, = np.where((df.kernel_size == ksz) &
                                    (df.kernel_stride == kstrd) &
                                    (df.pool_size == psz))
                    val_loss = np.median(df.iloc[idx,df.columns.get_loc('val_loss')].to_numpy())
                    if psz == 4 and ksz == 5 and kstrd == 1:
                        fid.write('\\blue{{{}}} & \\blue{{{}}} & \\blue{{{}}} & \\blue{{{:4d}}} & \\blue{{{:7.5f}}} \\\\\n'.
                                  format(ksz, kstrd, psz, out_sz, val_loss))
                    elif psz == 6 and ksz in (3,5) and kstrd == 1:
                        fid.write('\\green{{{}}} & \\green{{{}}} & \\green{{{}}} & \\green{{{:4d}}} & \\green{{{:7.5f}}} \\\\\n'.
                                  format(ksz, kstrd, psz, out_sz, val_loss))
                    else:
                        fid.write('{} & {} & {} & {:4d} & {:7.5f} \\\\\n'.
                                  format(ksz, kstrd, psz, out_sz, val_loss))
    fid.write('\\bottomrule\n')
    fid.write('\\end{tabular}\n')
    fid.write('\\end{center}\n')
    fid.write('\\end{table}\n')

## Figure S4
Not sure, maybe it'll be just for a reviewer...

In [ ]:
# full grid without varying compensator's inertia
# experiment_ID = 'f64bde90cab54d1ea770bb21f33c3ed1'
# full grid with variable compensator's inertia
experiment_ID = 'a40658acee3c4e419c0ee34d0c59f4df'
datafile = 'wide_grid_test_' + experiment_ID[:6]
df = pd.read_parquet(datafile + '.parquet.gz')

#####
fig,ax = plt.subplots(1, 1, figsize=(3,2.5))
x = [3.33, 5.33, 5.33, 3.33, 3.33]
y = [3.47, 3.47, 5.47, 5.47, 3.47]
ax.plot(x, y, '-', color=.7+np.zeros(3), lw=1)
ax.fill_between(x[:2], y[:2], y[2:4], fc=.8+np.zeros(3), alpha=0.5)

coeff = 200
for sz in (0.05,0.1,0.2,0.5):
    ax.scatter(0, 0, s=sz*coeff, c='k', label=f'{int(sz*100)}%')
im = ax.scatter(df['H_G02'], df['H_G03'], s=df['MAPE']*coeff, c=df['MAPE'],
           cmap='RdYlBu_r', vmin=0.0, vmax=0.6)
ax.legend(loc='best', frameon=False, bbox_to_anchor=(1.3,0.1))

cbar = plt.colorbar(im, shrink=0.8)
cbar.set_label('MAPE [%]')
ticks = np.r_[0 : 0.65 : 0.1]
cbar.set_ticks(ticks)
cbar.set_ticklabels([f'{tick*100:.0f}' for tick in ticks])

ax.set_xlim([1.2,10.8])
ax.set_ylim([1.2,10.8])
ticks = np.arange(2, 11, 2)
ax.set_xticks(ticks)
ax.set_yticks(ticks)
ax.set_xlabel(r'$H_{G2}$ [s]')
ax.set_ylabel(r'$H_{G3}$ [s]')
ax.text(1.5, 10.75, experiment_ID[:6], fontsize=fontsize+2)
sns.despine()
fig.tight_layout(pad=0.1)
fig.savefig(datafile + '.pdf')

## Figure S5
Effect of varying the damping coefficient on the estimation of momentum.

In [ ]:
# full grid without varying compensator's inertia
# experiment_ID = 'f64bde90cab54d1ea770bb21f33c3ed1'
# full grid with variable compensator's inertia
experiment_ID = 'a40658acee3c4e419c0ee34d0c59f4df'
experiments_path = '../experiments/neural_network'
network_pars = pickle.load(open(os.path.join(experiments_path, experiment_ID, 'parameters.pkl'), 'rb'))
area_ID = network_pars['area_IDs'][0]
data_dir = network_pars['data_dirs'][0].format(area_ID)

damping_data_file = 'var_D_area_1.npz'
force = False
if force or not os.path.isfile(damping_data_file):
    data_files = sorted(glob.glob(os.path.join('..', data_dir, 'ieee39_*_D*_*.h5')))
    D = np.array([float(re.findall('D=\d.\d', d)[0].split('=')[1]) for d in data_files])
    window_dur = 60,
    window_step = window_dur
    var_names = network_pars['var_names']
    x_train_mean = network_pars['x_train_mean']
    x_train_std = network_pars['x_train_std']
    data_mean = {var_name: x_train_mean[k] for k,var_name in enumerate(var_names)}
    data_std = {var_name: x_train_std[k] for k,var_name in enumerate(var_names)}
    Xf = []
    for data_file in data_files:
        t, _, _, X_slide, _ = load_data_slide([data_file], var_names, data_mean, data_std,
                                                   window_dur, window_step, add_omega_ref=False,
                                                   verbose=True)
        dt = np.diff(t[:2])[0]
        Xf.append({})
        for var_name in var_names:
            N_samples = X_slide[var_name].shape[1]
            tmp = fft(X_slide[var_name])
            tmp = 2.0 / N_samples * np.abs(tmp[:, :N_samples//2])
            Xf[-1][var_name] = tmp.mean(axis=0)
            F = fftfreq(N_samples, dt)[:N_samples//2]
    data = {'F': F, 'Xf': Xf, 'D': D, 'window_dur': window_dur, 'window_step': window_step,
            'var_names': var_names, 'x_train_mean': x_train_mean, 'x_train_std': x_train_std}
    np.savez_compressed(damping_data_file, **data)
else:
    data = np.load(damping_data_file, allow_pickle=True)
    D,F,Xf = data['D'], data['F'], data['Xf']
    var_names = data['var_names']

db = shelve.open(os.path.join(experiments_path, experiment_ID,
                              experiment_ID[:6]+'_Pfrac=0.1_KAD=2.0_KW=10.0.out'))
experiments = db['experiments'][9:9+len(D)]
db.close()

In [ ]:
dfs = []
for expt in experiments:
    damping = float(re.findall('\d.\d', expt['description'])[-1])
    M = expt['exact'][0]
    M_pred = np.squeeze(expt['prediction'])
    idx = np.where(np.logical_not(np.isnan(M_pred)))[0][0]
    df = pd.DataFrame(data={'D': damping, 'M': M, 'M_pred': M_pred[idx::60]})
    dfs.append(df)
df = pd.concat(dfs)
M = df.M.unique()[0]
df['MAPE'] = (df['M'] - df['M_pred']).abs() / df['M'] * 100
df_mean = df.groupby('D').mean()
df_mean

In [ ]:
palette = [[.8,.8,.8] for i in range(len(D))]
offset = np.array([[0.12,0.1], [0.1,0.05]])
space = [0.1, 0.09]
w = 1 - np.sum(offset[:,0])
h = 0.26
h_inset = 1 - np.sum(offset[:,1]) - 2*h - 2*space[1]
w_inset = (1 - np.sum(offset[:,0]) - space[0]) / 2
fig = plt.figure(figsize=(4,4))
ax = [
    plt.axes([offset[0,0], 1-offset[1,1]-h, w, h]),
    plt.axes([offset[0,0], offset[0,1], w, h]),
    plt.axes([offset[0,0], offset[0,1] + h + space[1]/1.7, w_inset, h_inset]),
    plt.axes([offset[0,0]+w_inset+space[0], offset[0,1] + h + space[1]/1.7, w_inset, h_inset])
]

xlim = [-0.5, len(D)-0.5]
ax[0].plot(xlim, [M, M], 'r', lw=2, zorder=-1)
sns.violinplot(x='D', y='M_pred', data=df, cut=0, inner='quartile',
               palette=palette, ax=ax[0], linewidth=1, zorder=0)
twin = ax[0].twinx()
twin.plot(df_mean['MAPE'], 'ko', ms=7, markerfacecolor='w', markeredgewidth=1.5)
ax[0].set_xlim(xlim)
ax[0].set_yticks(np.r_[0.2 : 0.25 : 0.01])
ax[0].set_xlabel('Damping')
ax[0].set_ylabel(r'M [GW$\cdot$s$^2$]')
ax[0].set_ylim([0.2, 0.24])
ax[0].set_yticks(np.linspace(0.2, 0.24, 5))
twin.set_ylim([1, 4.2])
twin.set_yticks(np.linspace(1, 4.2, 5))
twin.set_ylabel('MAPE [%]')
# ax[0].grid(which='major', axis='y', ls=':', lw=0.5, color=[.6,.6,.6])

xlim = [[1e-2,20], [1e-2,1e-1], [0.4,1.5]]
ylim = [[-60,0], [-20,0], [-30,-10]]
var_name = 'Vd_bus3'
cmap = plt.get_cmap('viridis', len(Xf))
for i,a in enumerate(ax[1:]):
    for k,xf in enumerate(Xf):
        a.plot(F, 20*np.log10(xf[var_name]), color=cmap(k), lw=1, label=f'D={k:.1f}')
    a.set_xscale('log')
    a.set_xlim(xlim[i])
    a.set_ylim(ylim[i])
    a.set_yticks(np.r_[ylim[i][0] : ylim[i][1]+5 : 20 if i == 0 else 10])
    a.grid(which='major', axis='x', lw=0.5, ls=':', color=[.6,.6,.6])
ax[1].set_xlabel('Frequency [Hz]')
ax[1].set_ylabel('Power [dB]')
ax[1].legend(loc='lower left', frameon=False)
xticks = np.array([0.01, 0.03, 0.1, 0.3, 1, 3, 10, 20])
ax[1].xaxis.set_major_locator(FixedLocator(xticks))
ax[1].xaxis.set_minor_locator(NullLocator())
ax[1].xaxis.set_major_formatter(FixedFormatter([f'{tick:g}' for tick in xticks]))
ax[2].set_ylabel('Power [dB]')
ax[2].xaxis.set_major_locator(FixedLocator(xticks[:3]))
ax[2].xaxis.set_minor_locator(NullLocator())
ax[2].xaxis.set_major_formatter(FixedFormatter([f'{tick:g}' for tick in xticks[:3]]))
xticks = np.logspace(np.log10(xlim[2][0]), np.log10(xlim[2][1]), 4)
ax[3].xaxis.set_major_locator(FixedLocator(xticks))
ax[3].xaxis.set_minor_locator(NullLocator())
ax[3].xaxis.set_major_formatter(FixedFormatter([f'{tick:.1f}' for tick in xticks]))

for side in 'right','top':
    for a in ax[1:]:
        a.spines[side].set_visible(False)
        
trans = mtransforms.ScaledTranslation(-0.45, 0, fig.dpi_scale_trans)
ax[0].text(0.0, 1.0, 'A', transform=ax[0].transAxes+trans, fontsize=10, va='bottom')
ax[2].text(0.0, 1.0, 'B', transform=ax[2].transAxes+trans, fontsize=10, va='bottom')

fig.savefig('var_D_area_1.pdf')